## BEX - Binance Ecosystem Index - Methodology Template

## Configuring packages

In [12]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [13]:
min_mcap =  0.2e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = 'binance-smart-chain'

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')
liquid_staking = pd.DataFrame(cg.get_coins_markets('usd',category='liquid-staking-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [14]:

results,liquidity = methodology_class.main(
    single_chain='binance-smart-chain',
    df_to_remove=[stablecoins,ctokens,atokens,liquid_staking]
    )
liquidity

Excluding beam-2, pricing data available only for 45 < 180 days
Note: plearn, marketcap data available only for 59 < 180 days
Excluding planet-token, pricing data available only for 177 < 180 days
Excluding gameai, pricing data available only for 84 < 180 days
Note: chex-token, marketcap data available only for 4 < 180 days
Note: magpie, marketcap data available only for 151 < 180 days
Note: the-big-five, marketcap data available only for 4 < 180 days
Note: domi, marketcap data available only for 5 < 180 days
0x6e88056e8376ae7709496ba64d37fa2f8015ce3e binance-smart-chain
0x644192291cc835a93d6330b24ea5f5fedd0eef9e binance-smart-chain
0xc03fbf20a586fa89c2a5f6f941458e1fbc40c661 binance-smart-chain
0xd1738eb733a636d1b8665f48bc8a24da889c2562 binance-smart-chain
0x7588df009c3d82378be6ab81f2108fa963c10fc8 binance-smart-chain
Dropping wrapped-beacon-eth because a price feed is unavailable
Dropping radiant-capital because a price feed is unavailable
Dropping enjinstarter because a price feed is

,spot price,delivery price,slippage,blockchain
id,,,,
matic-network,1.088902e+00,1.087914e+00,-0.000908,polygon-pos
binancecoin,4.150435e-03,4.145226e-03,-0.001255,binance-smart-chain
pancakeswap-token,3.931080e-01,3.919356e-01,-0.002982,binance-smart-chain
wrapped-beacon-eth,4.147380e-04,4.133263e-04,-0.003404,binance-smart-chain
radiant-capital,3.521466e+00,3.489601e+00,-0.009049,binance-smart-chain
...,...,...,...,...
joe,1.544252e-02,1.544917e-05,-0.999000,binance-smart-chain
velo,1.245597e-01,1.245641e-04,-0.999000,binance-smart-chain
perpetual-protocol,4.337395e-05,4.336700e-08,-0.999000,binance-smart-chain


## Show Results

In [15]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
binancecoin,BNB,37016039659,0.301697,77,BNB,binance-smart-chain
chainlink,Chainlink,9334526520,0.301697,77,0xf8a0bf9cf54bb92f17374d9e9a321e6a111a51bd,binance-smart-chain
matic-network,Polygon,8448364006,0.301697,77,MATIC,polygon-pos
pancakeswap-token,PancakeSwap,658876540,0.037849,10,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,binance-smart-chain
floki,FLOKI,413042189,0.023727,7,0xfb5b838b6cfeedc2873ab27866079ac55363d37e,binance-smart-chain
seedify-fund,Seedify.fund,248653390,0.014284,3,0x477bc8d23c634c154061869478bce96be6045d12,binance-smart-chain
baby-doge-coin,Baby Doge Coin,213909588,0.012288,3,0xc748673057861a797275cd8a068abb95a902e8de,binance-smart-chain
stargate-finance,Stargate Finance,117697181,0.006761,1,0xb0d502e938ed5f4df2e681fe6e419ff29631d62b,binance-smart-chain
